## CDK 環境の構築

### Install AWS CLI

In [ ]:
%%bash
cd /tmp
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip -oq awscliv2.zip
sudo ./aws/install --update

### Install Node.js

In [ ]:
%%bash
cd /tmp
curl -fsSL https://deb.nodesource.com/setup_lts.x | sudo -E bash - &&\
sudo apt install -y nodejs

### Install pnpm

In [ ]:
%%bash
sudo npm install -g pnpm
pnpm setup
source ~/.bashrc

### Install CDK

In [ ]:
%%bash
pnpm install -g aws-cdk

### CDK Bootstrap

In [ ]:
%%bash
cdk bootstrap

## CDK実行

### すべてデプロイ

In [ ]:
%%bash
pnpm run deploy

### 一部だけデプロイ

In [ ]:
%%bash
cdk list

In [ ]:
%%bash
cdk deploy ModerndataStack/glueStack --require-approval never

# ---

# もろもろ

## 変数初期化

In [ ]:
%pip install boto3

In [ ]:
import boto3

sts_client = boto3.client("sts")
identity = sts_client.get_caller_identity()

account = identity["Account"]
region = sts_client.meta.region_name

print(f"{account} - {region}")

## LakeFormation設定

### Default permissions for newly created databases and tables

権限がないと実行できません

In [ ]:
import boto3

lakeformation_client = boto3.client("lakeformation")

lakeformation_client.get_data_lake_settings()

lakeformation_client.put_data_lake_settings(
    CatalogId=account,
    DataLakeSettings={
        "CreateDatabaseDefaultPermissions": [],
        "CreateDatabaseDefaultPermissions": [],
    },
)

### Data Filter

#### 作成

In [ ]:
import boto3

lakeformation_client = boto3.client("lakeformation")

lakeformation_client.list_data_cells_filter()

filter_list = [
    {"Name": "only-jp", "Filter": "\"iso_code\" = 'JPN'"},
    {"Name": "only-us", "Filter": "\"iso_code\" = 'USA'"},
    {
        "Name": "only-eu",
        "Filter": "\"iso_code\" IN ('AUT','BEL','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','IRL','ITA','LVA','LTU','LUX','MLT','NLD','POL','PRT','ROU','SVK','SVN','ESP')",
    },
]

for target in filter_list:
    lakeformation_client.create_data_cells_filter(
        TableData={
            "TableCatalogId": account,
            "DatabaseName": "stage",
            "TableName": "world_cases_deaths_testing",
            "Name": target["Name"],
            "ColumnNames": [],
            "ColumnWildcard": {"ExcludedColumnNames": []},
            "RowFilter": {"FilterExpression": target["Filter"]},
        }
    )

#### 削除

In [ ]:
import boto3

lakeformation_client = boto3.client("lakeformation")

lakeformation_client.list_data_cells_filter()

filter_list = ["only-jp", "only-us", "only-eu"]

for target in filter_list:
    try:
        lakeformation_client.delete_data_cells_filter(
            TableCatalogId=account,
            DatabaseName="stage",
            TableName="world_cases_deaths_testing",
            Name=target,
        )
    except:
        pass

## Permission

### IAM_ALLOWED_PRINCIPALSをRevoke

In [ ]:
import boto3

lakeformation_client = boto3.client("lakeformation")

permission_list = lakeformation_client.list_permissions()

iam_allowed_principals_permission = list(
    filter(
        lambda x: x["Principal"]["DataLakePrincipalIdentifier"]
        == "IAM_ALLOWED_PRINCIPALS",
        permission_list["PrincipalResourcePermissions"],
    )
)

for permission in iam_allowed_principals_permission:
    lakeformation_client.revoke_permissions(
        CatalogId=account,
        Principal=permission["Principal"],
        Resource=permission["Resource"],
        Permissions=permission["Permissions"],
        PermissionsWithGrantOption=permission["PermissionsWithGrantOption"],
    )

while "NextToken" in permission_list:
    permission_list = lakeformation_client.list_permissions(
        NextToken=permission_list["NextToken"]
    )

    iam_allowed_principals_permission = list(
        filter(
            lambda x: x["Principal"]["DataLakePrincipalIdentifier"]
            == "IAM_ALLOWED_PRINCIPALS",
            permission_list["PrincipalResourcePermissions"],
        )
    )

    for permission in iam_allowed_principals_permission:
        lakeformation_client.revoke_permissions(
            CatalogId=account,
            Principal=permission["Principal"],
            Resource=permission["Resource"],
            Permissions=permission["Permissions"],
            PermissionsWithGrantOption=permission["PermissionsWithGrantOption"],
        )

### Stageデータベースに権限付与



In [ ]:
import boto3

lakeformation_client = boto3.client("lakeformation")

tables = ["world_cases_deaths_testing", "countrycode"]

target_list = [
  {'role_name': 'analyst-ja-role', 'data_cells_filter': 'only-jp'},
  {'role_name': 'analyst-us-role', 'data_cells_filter': 'only-us'},
  {'role_name': 'analyst-eu-role', 'data_cells_filter': 'only-eu'},
  {'role_name': 'RedshiftRole-ja', 'data_cells_filter': 'only-jp'},
  {'role_name': 'RedshiftRole-us', 'data_cells_filter': 'only-us'},
  {'role_name': 'RedshiftRole-eu', 'data_cells_filter': 'only-eu'},
  ]

for target in target_list:
  
  lakeformation_client.grant_permissions(
      CatalogId=account,
      Principal={
          "DataLakePrincipalIdentifier": f"arn:aws:iam::{account}:role/{target['role_name']}"
      },
      Resource={
          "DataCellsFilter": {
              "DatabaseName": "stage",
              "TableName": "world_cases_deaths_testing",
              "Name": f"{target['data_cells_filter']}",
          },
      },
      Permissions=["SELECT"],
  )

  lakeformation_client.grant_permissions(
      CatalogId=account,
      Principal={
          "DataLakePrincipalIdentifier": f"arn:aws:iam::{account}:role/{target['role_name']}"
      },
      Resource={
          "Table": {"DatabaseName": "stage", "Name": "countrycode"},
      },
      Permissions=["SELECT"],
  )


## Redshift

1. それぞれのユーザーにスイッチロールしてRedshiftクエリエディタV2にフェデレーションログインする。
1. 管理者権限でRedshiftクエリエディタV2にログインし、SQLを実行

### スイッチロールURL

In [ ]:
target_list = [
    {'workgroup': 'ja', 'redshift_role': 'RedshiftRole-ja', 'iam_role': 'analyst-ja-role'},
    {'workgroup': 'us', 'redshift_role': 'RedshiftRole-us', 'iam_role': 'analyst-us-role'},
    {'workgroup': 'eu', 'redshift_role': 'RedshiftRole-eu', 'iam_role': 'analyst-eu-role'},
 ]

for target in target_list:
    print(f'''
    https://signin.aws.amazon.com/switchrole?roleName={target['iam_role']}&account={account}
    ''')


print('''
    クエリエディタV2
    https://us-east-1.console.aws.amazon.com/sqlworkbench/home?region=us-east-1#/client
''')


### SQL

In [ ]:

target_list = [
    {'workgroup': 'ja', 'redshift_role': 'RedshiftRole-ja', 'iam_role': 'analyst-ja-role'},
    {'workgroup': 'us', 'redshift_role': 'RedshiftRole-us', 'iam_role': 'analyst-us-role'},
    {'workgroup': 'eu', 'redshift_role': 'RedshiftRole-eu', 'iam_role': 'analyst-eu-role'},
 ]

for target in target_list:

  print(f'''
  ---
  workgroup-{target['workgroup']}
  ---

  create external schema datalake
  from data catalog
  database 'stage'
  iam_role 'arn:aws:iam::{account}:role/{target['redshift_role']}'
  region 'us-east-1';

  select * from pg_user;
  GRANT ALL ON DATABASE dev to "IAMR:{target['iam_role']}";
  GRANT ALL ON SCHEMA "datalake" to "IAMR:{target['iam_role']}";

  ''')
      


## SiteWiseのコールドストレージの設定

### 有効化

In [ ]:
import boto3

sitewise_client = boto3.client("iotsitewise")

sitewise_client.describe_storage_configuration()

sitewise_client.put_storage_configuration(
    storageType="MULTI_LAYER_STORAGE",
    multiLayerStorage={
        "customerManagedS3Storage": {
            "s3ResourceArn": f"arn:aws:s3:::{account}-virginia-lakehouse-datalake-raw/iot-sitewise/",
            "roleArn": f"arn:aws:iam::{account}:role/sitewise-storage-role",
        }
    },
)

### 無効化

In [ ]:
import boto3

sitewise_client = boto3.client("iotsitewise")

sitewise_client.describe_storage_configuration()

sitewise_client.put_storage_configuration(
    storageType="SITEWISE_DEFAULT_STORAGE",
)

## S3バケットのクリーンアップ

### バケットのバージョニングを無効化

In [ ]:
import boto3

target_bucket_list = {
    "s3BucketAudit": f"{account}-virginia-audit",
    "athenaBucket": f"aws-athena-query-results-{account}-virginia",
    "s3BucketRaw": f"{account}-virginia-lakehouse-datalake-raw",
    "s3BucketStage": f"{account}-virginia-lakehouse-datalake-stage",
    "s3BucketJob": f"aws-glue-assets-{account}-virginia",
}

s3_client = boto3.client("s3")

bucket_list = s3_client.list_buckets()

for bucket in bucket_list["Buckets"]:
    bucket_name = bucket["Name"]

    if bucket_name in list(target_bucket_list.values()):
        bucket_versioning = s3_client.get_bucket_versioning(Bucket=bucket_name)

        if "Status" in bucket_versioning.keys():
            print(f'{bucket_name} - {bucket_versioning["Status"]}')
            s3_client.put_bucket_versioning(
                Bucket=bucket_name, VersioningConfiguration={"Status": "Suspended"}
            )
            bucket_versioning_check = s3_client.get_bucket_versioning(
                Bucket=bucket_name
            )
            print(f'{bucket_name} - {bucket_versioning_check["Status"]}')

### バケット削除

In [ ]:
%%bash
identity=$(aws sts get-caller-identity)
ACCOUNT=$(echo $identity | jq -r '.Account')
REGION=$(aws configure get region)

S3BUCKETAUDIT=${ACCOUNT}-virginia-audit
S3BUCKETRAW=${ACCOUNT}-virginia-lakehouse-datalake-raw
S3BUCKETSTAGE=${ACCOUNT}-virginia-lakehouse-datalake-stage
S3ATHENABUCKET=aws-athena-query-results-${ACCOUNT}-virginia

aws s3 rb --force s3://${S3BUCKETAUDIT}
aws s3 rb --force s3://${S3BUCKETRAW}
aws s3 rb --force s3://${S3BUCKETSTAGE}
aws s3 rb --force s3://${S3ATHENABUCKET}
